In [1]:
!pip3 install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 12.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == 

In [2]:
import pandas as pd
import numpy as np
import datasets
from transformers import T5Tokenizer

In [3]:
!unzip new_dataset.zip

Archive:  new_dataset.zip
  inflating: test.en                 
  inflating: test.fr                 
  inflating: train.fr                
  inflating: val.fr                  
  inflating: val.en                  
  inflating: train.en                


In [4]:
french_train_df = pd.read_csv("train.fr", sep='\t', header=None, names=["input_to_translate"])
english_train_df = pd.read_csv("train.en", sep='\t', header=None, names=["label"])
french_test_df = pd.read_csv("val.fr", sep='\t', header=None, names=["input_to_translate"])
english_test_df = pd.read_csv("val.en", sep='\t', header=None, names=["label"])

In [5]:
def generate_prompt(x):
    language_mapping = {"en":"English", "es":"Spanish", "de":"German", "fr": "French", "it":"Italian", "pt":"Portuguese"}
    source_text = x["input_to_translate"]
    language = x["language_pair"].split('_')[1]
    input_text = f"Translate French to {language_mapping[language]}: {source_text}"
    return input_text

In [6]:
training_features = pd.concat([french_train_df,english_train_df],axis=1)
training_features["language_pair"] = "fr_en"
training_features["ID"] = range(len(training_features))
training_features["prompt"] = training_features.apply(generate_prompt, axis=1)

testing_features = pd.concat([french_test_df,english_test_df],axis=1)
testing_features["language_pair"] = "fr_en"
testing_features["ID"] = range(len(testing_features))
testing_features["prompt"] = testing_features.apply(generate_prompt, axis=1)

In [7]:
training_na_rows = training_features.index[training_features['label'].isna()].to_numpy()
testing_na_rows = testing_features.index[testing_features['label'].isna()].to_numpy()
training_features = training_features.drop(training_features.index[training_na_rows])
testing_features = testing_features.drop(testing_features.index[testing_na_rows])

In [8]:
training_features.head(2)

,input_to_translate,label,language_pair,ID,prompt
0,pre-filled pen,Stylo pré-rempli (verre) (SureClick),fr_en,0,Translate French to English: pre-filled pen
1,Not applicable.,Sans objet.,fr_en,1,Translate French to English: Not applicable.


In [9]:
testing_features.head(2)

,input_to_translate,label,language_pair,ID,prompt
0,MANUFACTURING AUTHORISATION HOLDERS RESPONSIBL...,TITULAIRES DE L’AUTORISATION DE FABRICATION RE...,fr_en,0,Translate French to English: MANUFACTURING AUT...
1,EU Number,Numéro EU,fr_en,1,Translate French to English: EU Number


In [10]:
print(training_features.iloc[456]["prompt"])
print(training_features.iloc[456]["label"])
print(testing_features.iloc[456]["prompt"])
print(testing_features.iloc[456]["label"])

Translate French to English: Adult haemodialysis patients:
Patients adultes en hémodialyse:
Translate French to English: In conclusion, the benefit/ risk ratio is favourable for Crestor 5 or Crestor 10 mg as alternative start doses.
En conclusion, le rapport bénéfice/ risque de Crestor 5 et Crestor 10 mg (en doses initiales alternatives) est favorable.


In [11]:
train_ds_raw = datasets.Dataset.from_pandas(training_features)
train_ds_raw = train_ds_raw.filter(lambda x: x["label"] is not None and x["label"] != "")
test_ds_raw = datasets.Dataset.from_pandas(testing_features)
test_ds_raw = test_ds_raw.filter(lambda x: x["input_to_translate"] is not None and x["input_to_translate"] != "")

Filter:   0%|          | 0/811263 [00:00<?, ? examples/s]

Filter:   0%|          | 0/101218 [00:00<?, ? examples/s]

In [12]:
tokenizer = T5Tokenizer.from_pretrained("t5-small")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [13]:
tokenized_source_training = train_ds_raw.map(
    lambda x: tokenizer(x["prompt"], truncation=True),
    batched=True, remove_columns=['ID', 'input_to_translate', 'label', 'language_pair', 'prompt'])

tokenized_source_testing = test_ds_raw.map(
    lambda x: tokenizer(x["prompt"], truncation=True),
    batched=True, remove_columns=['ID', 'input_to_translate', 'label', 'language_pair', 'prompt'])

source_lengths_training = [len(x) for x in tokenized_source_training["input_ids"]]
source_lengths_testing = [len(x) for x in tokenized_source_testing["input_ids"]]

Map:   0%|          | 0/811263 [00:00<?, ? examples/s]

Map:   0%|          | 0/99097 [00:00<?, ? examples/s]

In [15]:
tokenized_target_training = train_ds_raw.map(
    lambda x: tokenizer(x["label"], truncation=True),
    batched=True, remove_columns=['ID', 'input_to_translate', 'label', 'language_pair', 'prompt'])
target_lengths_training = [len(x) for x in tokenized_target_training["input_ids"]]

tokenized_target_testing = test_ds_raw.map(
    lambda x: tokenizer(x["label"], truncation=True),
    batched=True, remove_columns=['ID', 'input_to_translate', 'label', 'language_pair', 'prompt'])
target_lengths_testing = [len(x) for x in tokenized_target_testing["input_ids"]]

Map:   0%|          | 0/811263 [00:00<?, ? examples/s]

Map:   0%|          | 0/99097 [00:00<?, ? examples/s]

In [19]:
print(f"Train source Min source length: {min(source_lengths_training)}")
print(f"Train source Max source length: {max(source_lengths_training)}")
print(f"Train source 99% source length: {int(np.percentile(source_lengths_training, 99))}")
print(f"Train target Min target length: {min(target_lengths_training)}")
print(f"Train target Max target length: {max(target_lengths_training)}")
print(f"Train target 99% target length: {int(np.percentile(target_lengths_training, 99))}")

print(f"Test source Min source length: {min(source_lengths_testing)}")
print(f"Test source Max source length: {max(source_lengths_testing)}")
print(f"Test source 99% source length: {int(np.percentile(source_lengths_testing, 99))}")
print(f"Test target Min target length: {min(target_lengths_testing)}")
print(f"Test target Max target length: {max(target_lengths_testing)}")
print(f"Test target 99% target length: {int(np.percentile(target_lengths_testing, 99))}")

Train source Min source length: 8
Train source Max source length: 512
Train source 99% source length: 92
Train target Min target length: 2
Train target Max target length: 512
Train target 99% target length: 125
Test source Min source length: 8
Test source Max source length: 512
Test source 99% source length: 92
Test target Min target length: 2
Test target Max target length: 512
Test target 99% target length: 125


In [20]:
# reference: https://www.philschmid.de/fine-tune-flan-t5-deepspeed
def preprocess_function(sample, padding="max_length"):

    # tokenize inputs
    model_inputs = tokenizer(sample["prompt"], max_length=128, padding=padding, truncation=True)

    # Tokenize targets with the `text_target` keyword argument
    labels = tokenizer(text_target=sample["label"], max_length=128, padding=padding, truncation=True)

    # If we are padding here, replace all tokenizer.pad_token_id in the labels by -100 when we want to ignore
    # padding in the loss.
    if padding == "max_length":
        labels["input_ids"] = [
            [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
        ]

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


In [21]:
tokenized_train_ds = train_ds_raw.map(
    preprocess_function, batched=True,
    remove_columns=['ID', 'input_to_translate', 'label', 'language_pair', 'prompt'])
tokenized_test_ds = test_ds_raw.map(
    preprocess_function, batched=True,
    remove_columns=['ID', 'input_to_translate', 'label', 'language_pair', 'prompt'])

Map:   0%|          | 0/811263 [00:00<?, ? examples/s]

Map:   0%|          | 0/99097 [00:00<?, ? examples/s]

In [22]:
tokenized_train_ds.save_to_disk("tokenized_train_ds/train")
tokenized_test_ds.save_to_disk("tokenized_train_ds/eval")

Saving the dataset (0/3 shards):   0%|          | 0/811263 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/99097 [00:00<?, ? examples/s]

In [25]:
!zip -r medical_clean_tokenized.zip tokenized_train_ds

  adding: tokenized_train_ds/ (stored 0%)
  adding: tokenized_train_ds/train/ (stored 0%)
  adding: tokenized_train_ds/train/data-00002-of-00003.arrow (deflated 93%)
  adding: tokenized_train_ds/train/data-00000-of-00003.arrow (deflated 93%)
  adding: tokenized_train_ds/train/data-00001-of-00003.arrow (deflated 93%)
  adding: tokenized_train_ds/train/dataset_info.json (deflated 69%)
  adding: tokenized_train_ds/train/state.json (deflated 55%)
  adding: tokenized_train_ds/eval/ (stored 0%)
  adding: tokenized_train_ds/eval/dataset_info.json (deflated 69%)
  adding: tokenized_train_ds/eval/state.json (deflated 39%)
  adding: tokenized_train_ds/eval/data-00000-of-00001.arrow (deflated 93%)


In [26]:
from google.colab import files
files.download('medical_clean_tokenized.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>